In [3]:
from window_generator import WindowGenerator
import numpy as np
import pandas as pd
import kerastuner as kt
from fredapi import Fred
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

import math
import keras
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import save_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from kerastuner.tuners import RandomSearch
import tensorflow as tf
from kerastuner.engine.hyperparameters import HyperParameters
import warnings
warnings.simplefilter("ignore", UserWarning)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
raw_df = pd.read_csv("../data/feature_selected_data.csv", index_col=0, parse_dates=[0])

In [ ]:
df = raw_df.iloc[10:].copy()
del raw_df

In [ ]:
column_indices = {name: i for i, name in enumerate(df.columns)}
n = len(vix)
n_train = int(n*0.7)
n_val = int(n*0.9)
train_df = df[0:n_train]
val_df = df[n_train:n_val]
test_df = df[n_val:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
input_width = 30

w = WindowGenerator(input_width=input_width, 
                    label_width=1,
                    shift=1,
                    train_df=train_df,
                    val_df=val_df,
                    test_df=test_df,
                    n_splits=5,
                    train_splits=3,
                    test_splits=1, 
                    scaler=MinMaxScaler)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import kerastuner as kt

In [ ]:
from keras import backend as backend

def qlike(y_true, y_pred):
  return tf.math.log(y_pred) + (y_true / y_pred)

def build_model(hp):
    backend.clear_session()
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit', min_value=32, max_value=256, step=32), return_sequences=True, input_shape=(input_width, 1))) #(timesteps, features)
    for i in range(hp.Int('n_layers', 1, 2)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units', min_value=32, max_value=256, step=32), return_sequences=True))
    model.add(LSTM(hp.Int('layer_2_neurons', min_value=32, max_value=64, step=32)))
    model.add(Dropout(hp.Float('Dropout_rate', min_value=0.1, max_value=0.3, step=0.1)))
    model.add(Dense(1, activation="relu"))
    #model.add(Dense(1, activation=hp.Choice('dense_activation', values=['relu'], default='relu')))
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    metrics = ["mse", "mae", "mape", tf.keras.metrics.RootMeanSquaredError(), qlike]
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=learning_rate), metrics = metrics)
    return model

[Bayesian Optimization for hyperparameter search](https://arxiv.org/pdf/1806.10282.pdf)

[Keras tuner oracles](https://keras.io/api/keras_tuner/oracles/)

[Keras tuner API](https://keras.io/api/keras_tuner/)

In [ ]:
class CVTuner(kt.engine.tuner.Tuner):
    def run_trial(self, trial, window, *args, **kwargs):
        val_mse_list = []
        ###
        #batch_size = trial.hyperparameters.Int('batch_size', 8, 64, step=8)
        #epochs = trial.hyperparameters.Int('epochs', 10, 100, step=10)

        folds = window.np_folds
        for x_train, y_train, x_test, y_test in folds:
          model = self.hypermodel.build(trial.hyperparameters)
          model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test), verbose=0)
          val_loss, val_mse, val_mae, val_mape, val_rmse, val_qlike = model.evaluate(x_test, y_test)
          val_mse_list.append(val_loss)
          #self.save_model(trial.trial_id, model)
          del model
        del folds
        
        self.oracle.update_trial(trial.trial_id, {'val_mse': np.mean(val_mse_list),
                                                    'val_mse_std': np.std(val_mse_list)})


In [ ]:
tuner = CVTuner(oracle=kt.oracles.BayesianOptimizationOracle(objective='val_mse', max_trials=100), hypermodel=build_model, directory="baseline", project_name="lstm", overwrite=False)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_mse', patience=5)

tuner.search(window=w, callbacks=[stop_early])


In [ ]:
tuner.get_best_hyperparameters()[0].values